## This notebook performs the data preparation for the common regression dataset

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
# This is a kaggle dataset - https://www.kaggle.com/lepchenkov/usedcarscatalog
df = pd.read_csv('../../data/cars.csv')
df.columns

Index(['manufacturer_name', 'model_name', 'transmission', 'color',
       'odometer_value', 'year_produced', 'engine_fuel', 'engine_has_gas',
       'engine_type', 'engine_capacity', 'body_type', 'has_warranty', 'state',
       'drivetrain', 'price_usd', 'is_exchangeable', 'location_region',
       'number_of_photos', 'up_counter', 'feature_0', 'feature_1', 'feature_2',
       'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7',
       'feature_8', 'feature_9', 'duration_listed'],
      dtype='object')

In [3]:
df.head(5)

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,...,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,...,True,True,True,False,True,False,True,True,True,16
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,...,True,False,False,True,True,False,False,False,True,83
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,...,True,False,False,False,False,False,False,True,True,151
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,...,False,False,False,False,False,False,False,False,False,86
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,...,True,False,True,True,False,False,False,False,True,7


In [4]:
# Drop the columns that are definitely not planned to be used with the regression models 
# This is certainly subject to change, and further columns may be removed
#
# During the building and testing of the regression models, the columns transmission, engine_type, drivetrain were also dropped
df.drop(['transmission', 'engine_fuel', 'engine_has_gas', 'engine_type', 'has_warranty', 'body_type', 'state', 'drivetrain', 'is_exchangeable', 'location_region', 'up_counter', 'number_of_photos', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'duration_listed'], axis=1, inplace=True)

# A second round of feature reduction (additionally removed color and engine_capacity) to observe changes to model performance
# df.drop(['transmission', 'color', 'engine_fuel', 'engine_has_gas', 'engine_type', 'engine_capacity', 'has_warranty', 'body_type', 'state', 'drivetrain', 'is_exchangeable', 'location_region', 'up_counter', 'number_of_photos', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'duration_listed'], axis=1, inplace=True)
df.head(5)

,manufacturer_name,model_name,color,odometer_value,year_produced,engine_capacity,price_usd
0,Subaru,Outback,silver,190000,2010,2.5,10900.00
1,Subaru,Outback,blue,290000,2002,3.0,5000.00
2,Subaru,Forester,red,402000,2001,2.5,2800.00
3,Subaru,Impreza,blue,10000,1999,3.0,9999.00
4,Subaru,Legacy,black,280000,2001,2.5,2134.11


In [5]:
# get an overview of the dataset

df.describe(include='all')

,manufacturer_name,model_name,color,odometer_value,year_produced,engine_capacity,price_usd
count,38531,38531,38531,38531.000000,38531.000000,38521.000000,38531.000000
unique,55,1118,12,NaN,NaN,NaN,NaN
top,Volkswagen,Passat,black,NaN,NaN,NaN,NaN
freq,4243,1423,7705,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,248864.638447,2002.943734,2.055161,6639.971021
std,NaN,NaN,NaN,136072.376530,8.065731,0.671178,6428.152018
min,NaN,NaN,NaN,0.000000,1942.000000,0.200000,1.000000
25%,NaN,NaN,NaN,158000.000000,1998.000000,1.600000,2100.000000
50%,NaN,NaN,NaN,250000.000000,2003.000000,2.000000,4800.000000
75%,NaN,NaN,NaN,325000.000000,2009.000000,2.300000,8990.000000


In [6]:
# Check the data by feature for missing values

df.isnull().sum()

manufacturer_name     0
model_name            0
color                 0
odometer_value        0
year_produced         0
engine_capacity      10
price_usd             0
dtype: int64

In [7]:
# Let's see what these records look like

print(df[df['engine_capacity'].isnull()])

      manufacturer_name model_name   color  odometer_value  year_produced  \
8782               Fiat        500  orange           27000           2013   
9048               Fiat        500  orange           49000           2014   
24226         Chevrolet       Volt  silver          168000           2013   
25943            Nissan       Leaf   white           57357           2015   
26203            Nissan       Leaf    blue           97400           2011   
26222            Nissan       Leaf   white           50000           2014   
26582            Nissan       Leaf   black           84000           2014   
26914            Nissan       Leaf   black           84500           2013   
27554               BMW         i3   white           54150           2015   
29590               BMW         i3   other           67000           2018   

       engine_capacity  price_usd  
8782               NaN    14900.0  
9048               NaN    16000.0  
24226              NaN    12950.0  
25943   

In [8]:
# As we would expect, it doesn't make sense for an electric car to have a value for engine_capacity.
# For this study, we can tollerate dropping these records, however this may not be appropriate for 
# other datasets which could contain many more electric cars. In that case, we would likely prefer
# to remove the engine_capacity feature.

df = df.dropna()

In [9]:
# Instantiate new dataframes, so we can keep the original one as is for future reference, before encoding
# and scaling the data

df_X = pd.DataFrame(df)

In [10]:
# One-hot encode the categorical (nominal) features, and drop the original columns.
# For now, we'll encode all the nominal features, but we may not use all of them when building the 
# regression models

df_X = pd.concat([df_X, pd.get_dummies(df_X[['manufacturer_name', 'model_name', 'color']])], axis=1)
df_X = df_X.drop(['manufacturer_name', 'model_name', 'color'], axis=1)

# Required for the second round of feature reduction 
# df_X = pd.concat([df_X, pd.get_dummies(df_X[['manufacturer_name', 'model_name']])], axis=1)
# df_X = df_X.drop(['manufacturer_name', 'model_name'], axis=1)
df_X.head(5)

,odometer_value,year_produced,engine_capacity,price_usd,manufacturer_name_Acura,manufacturer_name_Alfa Romeo,manufacturer_name_Audi,manufacturer_name_BMW,manufacturer_name_Buick,manufacturer_name_Cadillac,...,color_brown,color_green,color_grey,color_orange,color_other,color_red,color_silver,color_violet,color_white,color_yellow
0,190000,2010,2.5,10900.00,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,290000,2002,3.0,5000.00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,402000,2001,2.5,2800.00,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,10000,1999,3.0,9999.00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,280000,2001,2.5,2134.11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# As mentioned in Chapter 10 of Python Machine Learning the linear regression implementation in 
# scikit-learn may work better with unstandardized variables. We have made a tests with both
# scaled and unscaled values, and the unscaled values certainly resulted in a better model

# x_scaler = MinMaxScaler()
# df_X[['odometer_value', 'year_produced', 'engine_capacity']] = x_scaler.fit_transform(df_X[['odometer_value', 'year_produced', 'engine_capacity']])

# Required for the second round of feature reduction
# df_X[['odometer_value', 'year_produced']] = x_scaler.fit_transform(df_X[['odometer_value', 'year_produced']])
df_X.head(5)

,odometer_value,year_produced,engine_capacity,price_usd,manufacturer_name_Acura,manufacturer_name_Alfa Romeo,manufacturer_name_Audi,manufacturer_name_BMW,manufacturer_name_Buick,manufacturer_name_Cadillac,...,color_brown,color_green,color_grey,color_orange,color_other,color_red,color_silver,color_violet,color_white,color_yellow
0,190000,2010,2.5,10900.00,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,290000,2002,3.0,5000.00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,402000,2001,2.5,2800.00,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,10000,1999,3.0,9999.00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,280000,2001,2.5,2134.11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# As mentioned above, unscaled values resulted in a better model

# y_scaler = MinMaxScaler()
# y = y_scaler.fit_transform(df_y)

In [13]:
# df.to_csv('../../data/cleanedCars.csv', index=False, header=False)
df_X.to_csv('../../data/cleanedCars.csv', index=False)